# EDA Notebook

In [2]:
import os

import polars as pl
import polars.selectors as cs
import pandas as pd
from pandas_dq import dq_report, Fix_DQ
from summarytools import dfSummary

## Load data

In [5]:
data_dir = '/Users/zacklarsen/Documents/Projects/kaggle-wids-datathon-2020/data/raw'

In [6]:
training_v2 = pl.read_csv(os.path.join(data_dir, 'training_v2.csv'), infer_schema_length=10000)
unlabeled = pl.read_csv(os.path.join(data_dir, 'unlabeled.csv'), infer_schema_length=10000)
sample_submission = pl.read_csv(os.path.join(data_dir, 'samplesubmission.csv'))
solution_template = pl.read_csv(os.path.join(data_dir, 'solution_template.csv'))
data_dictionary = pl.read_csv(os.path.join(data_dir, 'WiDS Datathon 2020 Dictionary.csv'))

# Inspect

In [7]:
solution_template.head()

encounter_id,hospital_death
i64,str
2,null
5,null
7,null
8,null
10,null


In [8]:
sample_submission.head()

encounter_id,hospital_death
i64,f64
2,0.5
5,0.2
7,0.001


In [9]:
training_v2.shape

(91713, 186)

In [11]:
unlabeled.shape

(39308, 186)

In [49]:
training_v2.head()

encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,…,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
i64,i64,i64,i64,str,str,i64,str,str,str,str,str,i64,str,str,f64,i64,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
66154,25312,118,0,"""68""","""22.73""",0,"""Caucasian""","""M""","""180.3""","""Floor""","""Floor""",92,"""admit""","""CTICU""",0.541667,0,"""73.9""","""2.3""","""113""","""502.01""",0,"""0""","""0.4""","""31""","""2.51""","""NA""","""3""","""6""","""0""","""4""","""168""","""118""","""27.4""","""0""","""40""","""NA""",…,"""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""0.1""","""0.05""","""0""","""0""","""1""","""0""","""0""","""0""","""0""","""0""","""Sepsis""","""Cardiovascular…"
114252,59342,81,0,"""77""","""27.42""",0,"""Caucasian""","""F""","""160""","""Floor""","""Floor""",90,"""admit""","""Med-Surg ICU""",0.927778,0,"""70.2""","""NA""","""108""","""203.01""",0,"""0""","""NA""","""9""","""0.56""","""1""","""1""","""3""","""0""","""1""","""145""","""120""","""36.9""","""0""","""46""","""37""",…,"""3.5""","""557""","""557""","""4.2""","""4.2""","""145""","""145""","""12.7""","""12.7""","""37""","""37""","""7.45""","""7.45""","""51""","""51""","""54.8""","""51""","""37""","""37""","""7.45""","""7.45""","""51""","""51""","""51""","""51""","""0.47""","""0.29""","""0""","""0""","""1""","""0""","""0""","""0""","""0""","""0""","""Respiratory""","""Respiratory"""
119783,50777,118,0,"""25""","""31.95""",0,"""Caucasian""","""F""","""172.7""","""Emergency Depa…","""Accident & Eme…",93,"""admit""","""Med-Surg ICU""",0.000694,0,"""95.3""","""NA""","""122""","""703.03""",0,"""0""","""NA""","""NA""","""NA""","""NA""","""3""","""6""","""0""","""5""","""NA""","""102""","""NA""","""0""","""68""","""NA""",…,"""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""Metabolic""","""Metabolic"""
79267,46918,118,0,"""81""","""22.64""",1,"""Caucasian""","""F""","""165.1""","""Operating Room…","""Operating Room…",92,"""admit""","""CTICU""",0.000694,0,"""61.7""","""NA""","""203""","""1206.03""",1,"""0""","""NA""","""NA""","""NA""","""0.6""","""4""","""6""","""0""","""5""","""185""","""114""","""25.9""","""1""","""60""","""30""",…,"""NA""","""43""","""43""","""NA""","""NA""","""NA""","""NA""","""8.8""","""8.8""","""37""","""27""","""7.44""","""7.34""","""337""","""102""","

In [10]:
training_v2.select("icu_admit_type")

ColumnNotFoundError: icu_admit_type

Error originated just after this operation:
DF ["encounter_id", "patient_id", "hospital_id", "hospital_death"]; PROJECT */186 COLUMNS; SELECTION: "None"

In [119]:
training_v2.select("icu_stay_type")

icu_stay_type
str
"""admit"""
"""admit"""
"""admit"""
"""admit"""
"""admit"""
"""admit"""
"""admit"""
"""admit"""
"""admit"""


In [50]:
unlabeled.head(5)

encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,…,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
i64,i64,i64,str,str,str,i64,str,str,str,str,str,i64,str,str,f64,i64,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2,127112,7,null,"""56""","""21.10243942""",0,"""Caucasian""","""M""","""180.3""","""Emergency Depa…","""Accident & Eme…",1105,"""admit""","""Neuro ICU""",0.104167,0,"""68.6""","""3.7""","""301""","""405.01""",0,"""0""","""NA""","""19""","""0.67""","""NA""","""2""","""5""","""0""","""1""","""276""","""105""","""42""","""0""","""57""","""NA""",…,"""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""0.13""","""0.08""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""Neurological""","""Neurologic"""
5,130737,178,null,"""NA""","""19.50059453""",1,"""Caucasian""","""F""","""145""","""Operating Room…","""Operating Room…",1043,"""admit""","""Med-Surg ICU""",0.230556,0,"""41""","""NA""","""217""","""1502.02""",1,"""0""","""NA""","""NA""","""NA""","""NA""","""4""","""6""","""0""","""4""","""NA""","""60""","""NA""","""0""","""53""","""NA""",…,"""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""0.09""","""0.03""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""Neurological""","""Neurologic"""
7,101458,86,null,"""66""","""32.51859636""",1,"""African Americ…","""M""","""170.2""","""Operating Room…","""Operating Room…",1035,"""admit""","""CCU-CTICU""",0.140972,0,"""94.2""","""NA""","""302""","""1208.06""",1,"""0""","""NA""","""22""","""1.84""","""NA""","""2""","""5""","""0""","""1""","""81""","""124""","""39.2""","""0""","""115""","""NA""",…,"""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""0.03""","""0.02""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""Cardiovascular…","""Cardiovascular…"
8,48067,163,null,"""64""","""20.7985195""",0,"""Caucasian""","""M""","""182.8""","""Operating Room…","""Operating Room…",1078,"""admit""","""Med-Surg ICU""",0.002083,0,"""69.5""","""1.9""","""303""","""211.03""",0,"""0""","""0.8""","""24""","""0.75""","""0.7""","""4""","""6""","""0""","""1""","""122""","""111""","""32""","""1""","""48""","""36""",…,"""NA""","""248""","""248""","""4""","""4""","""133""","""133""","""9.7""","""

In [51]:
data_dictionary.head(5)

Category,Variable Name,Unit of Measure,Data Type,Description,Example
str,str,str,str,str,str
"""identifier""","""encounter_id""","""None""","""integer""","""Unique identif…","""None"""
"""identifier""","""hospital_id""","""None""","""integer""","""Unique identif…","""None"""
"""identifier""","""patient_id""","""None""","""integer""","""Unique identif…","""None"""
"""demographic""","""hospital_death…","""None""","""binary""","""Whether the pa…","""0"""
"""demographic""","""age""","""Years""","""numeric""","""The age of the…","""None"""


In [123]:
(
    data_dictionary
    .with_columns(
        pl.when(pl.col("Variable Name").str.ends_with("_id"))
        .then(pl.lit("string"))
        .otherwise(pl.col("Variable Name"))
    )
)

Category,Variable Name,Unit of Measure,Data Type,Description,Example,literal
str,str,str,str,str,str,str
"""identifier""","""encounter_id""","""None""","""integer""","""Unique identif…","""None""","""string"""
"""identifier""","""hospital_id""","""None""","""integer""","""Unique identif…","""None""","""string"""
"""identifier""","""patient_id""","""None""","""integer""","""Unique identif…","""None""","""string"""
"""demographic""","""hospital_death…","""None""","""binary""","""Whether the pa…","""0""","""hospital_death…"
"""demographic""","""age""","""Years""","""numeric""","""The age of the…","""None""","""age"""
"""demographic""","""bmi""","""kilograms/metr…","""string""","""The body mass …","""21.5""","""bmi"""
"""demographic""","""elective_surge…","""None""","""binary""","""Whether the pa…","""0""","""elective_surge…"
"""demographic""","""ethnicity""","""None""","""string""","""The common nat…","""Caucasian""","""ethnicity"""
"""demographic""","""gender""","""None""","""string""","""The genotypica…","""F""","""gender"""


In [129]:
data_dictionary_cols = data_dictionary.select("Variable Name").to_series().to_list()
data_dictionary_cols[:5]

['encounter_id', 'hospital_id', 'patient_id', 'hospital_death', 'age']

In [132]:
training_v2_cols = training_v2.columns
training_v2_cols[:5]

['encounter_id', 'patient_id', 'hospital_id', 'hospital_death', 'age']

In [134]:
set(data_dictionary_cols).symmetric_difference(set(training_v2_cols))

{'icu_admit_type', 'pred'}

## Polars Value Counts

In [52]:
(
    training_v2
    .select(cs.string().approx_n_unique())
    .melt()
)

variable,value
str,u32
"""age""",74
"""bmi""",34835
"""ethnicity""",7
"""gender""",3
"""height""",399
"""hospital_admit…",16
"""icu_admit_sour…",6
"""icu_stay_type""",3
"""icu_type""",8


In [53]:
(
    training_v2
    .select(cs.string().n_unique())
    .melt()
)

variable,value
str,u32
"""age""",75
"""bmi""",34889
"""ethnicity""",7
"""gender""",3
"""height""",402
"""hospital_admit…",16
"""icu_admit_sour…",6
"""icu_stay_type""",3
"""icu_type""",8


## Polars Types

In [65]:
(
    data_dictionary
    .select(pl.col(["Variable Name", "Data Type"]))
)

Variable Name,Data Type
str,str
"""encounter_id""","""integer"""
"""hospital_id""","""integer"""
"""patient_id""","""integer"""
"""hospital_death…","""binary"""
"""age""","""numeric"""
"""bmi""","""string"""
"""elective_surge…","""binary"""
"""ethnicity""","""string"""
"""gender""","""string"""


In [114]:
(
    data_dictionary
    .select(pl.col(["Data Type"]))
    .unique()
)

Data Type
str
"""integer"""
"""numeric"""
"""string"""
"""binary"""


In [115]:
string_cols = (
    data_dictionary
    .select(pl.col(["Variable Name", "Data Type"]))
    .filter(pl.col("Data Type") == "string")
    .select(pl.col("Variable Name"))
    .to_series()
    .to_list()
)

string_cols

['bmi',
 'ethnicity',
 'gender',
 'hospital_admit_source',
 'icu_admit_source',
 'icu_admit_type',
 'icu_stay_type',
 'icu_type',
 'apache_2_diagnosis',
 'apache_3j_diagnosis',
 'apache_3j_bodysystem',
 'apache_2_bodysystem']

In [116]:
int_cols = (
    data_dictionary
    .select(pl.col(["Variable Name", "Data Type"]))
    .filter(pl.col("Data Type") == "integer")
    .select(pl.col("Variable Name"))
    .to_series()
    .to_list()
)

int_cols

['encounter_id',
 'hospital_id',
 'patient_id',
 'icu_id',
 'gcs_eyes_apache',
 'gcs_motor_apache',
 'gcs_verbal_apache']

In [117]:
numeric_cols = (
    data_dictionary
    .select(pl.col(["Variable Name", "Data Type"]))
    .filter(pl.col("Data Type") == "numeric")
    .select(pl.col("Variable Name"))
    .to_series()
    .to_list()
)

numeric_cols

['age',
 'height',
 'pre_icu_los_days',
 'weight',
 'albumin_apache',
 'bilirubin_apache',
 'bun_apache',
 'creatinine_apache',
 'fio2_apache',
 'glucose_apache',
 'heart_rate_apache',
 'hematocrit_apache',
 'map_apache',
 'paco2_apache',
 'paco2_for_ph_apache',
 'pao2_apache',
 'ph_apache',
 'resprate_apache',
 'sodium_apache',
 'temp_apache',
 'urineoutput_apache',
 'wbc_apache',
 'd1_diasbp_invasive_max',
 'd1_diasbp_invasive_min',
 'd1_diasbp_max',
 'd1_diasbp_min',
 'd1_diasbp_noninvasive_max',
 'd1_diasbp_noninvasive_min',
 'd1_heartrate_max',
 'd1_heartrate_min',
 'd1_mbp_invasive_max',
 'd1_mbp_invasive_min',
 'd1_mbp_max',
 'd1_mbp_min',
 'd1_mbp_noninvasive_max',
 'd1_mbp_noninvasive_min',
 'd1_resprate_max',
 'd1_resprate_min',
 'd1_spo2_max',
 'd1_spo2_min',
 'd1_sysbp_invasive_max',
 'd1_sysbp_invasive_min',
 'd1_sysbp_max',
 'd1_sysbp_min',
 'd1_sysbp_noninvasive_max',
 'd1_sysbp_noninvasive_min',
 'd1_temp_max',
 'd1_temp_min',
 'h1_diasbp_invasive_max',
 'h1_diasbp_inva

In [70]:
binary_cols = (
    data_dictionary
    .select(pl.col(["Variable Name", "Data Type"]))
    .filter(pl.col("Data Type") == "binary")
    .select(pl.col("Variable Name"))
    .to_series()
    .to_list()
)

binary_cols

['hospital_death',
 'elective_surgery',
 'readmission_status',
 'apache_post_operative',
 'arf_apache',
 'gcs_unable_apache',
 'intubated_apache',
 'ventilated_apache',
 'aids',
 'cirrhosis',
 'diabetes_mellitus',
 'hepatic_failure',
 'immunosuppression',
 'leukemia',
 'lymphoma',
 'solid_tumor_with_metastasis']

In [ ]:
binary_cols

In [78]:
(
    training_v2
    .select(binary_cols)
    .glimpse()
)

Rows: 91713
Columns: 16
$ hospital_death              <i64> 0, 0, 0, 0, 0, 0, 0, 0, 1, 0
$ elective_surgery            <i64> 0, 0, 0, 1, 0, 0, 0, 0, 0, 0
$ readmission_status          <i64> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
$ apache_post_operative       <i64> 0, 0, 0, 1, 0, 0, 0, 0, 0, 0
$ arf_apache                  <str> '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'
$ gcs_unable_apache           <str> '0', '0', '0', '0', 'NA', '0', '0', '0', '0', '0'
$ intubated_apache            <str> '0', '0', '0', '1', '0', '0', '1', '0', '0', '0'
$ ventilated_apache           <str> '0', '1', '0', '1', '0', '0', '1', '1', '1', '0'
$ aids                        <str> '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'
$ cirrhosis                   <str> '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'
$ diabetes_mellitus           <str> '1', '1', '0', '0', '0', '1', '1', '0', '0', '0'
$ hepatic_failure             <str> '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'
$ immunosuppression           <str>

In [112]:
(
    training_v2
    .with_columns(
        pl.when(pl.col(pl.Utf8) == "NA")
        .then(pl.lit(None))
        .otherwise(pl.col(pl.Utf8))
        .name.keep()
    )
    .with_columns(pl.col(binary_cols).cast(pl.Int8))
    .with_columns(pl.col(binary_cols).cast(pl.Boolean))
    .glimpse()
)

Rows: 91713
Columns: 186
$ encounter_id                   <i64> 66154, 114252, 119783, 79267, 92056, 33181, 82208, 120995, 80471, 42871
$ patient_id                     <i64> 25312, 59342, 50777, 46918, 34377, 74489, 49526, 50129, 10577, 90749
$ hospital_id                    <i64> 118, 81, 118, 118, 33, 83, 83, 33, 118, 118
$ hospital_death                <bool> False, False, False, False, False, False, False, False, True, False
$ age                            <str> '68', '77', '25', '81', '19', '67', '59', '70', '45', '50'
$ bmi                            <str> '22.73', '27.42', '31.95', '22.64', None, '27.56', '57.45', None, None, '25.71'
$ elective_surgery              <bool> False, False, False, True, False, False, False, False, False, False
$ ethnicity                      <str> 'Caucasian', 'Caucasian', 'Caucasian', 'Caucasian', 'Caucasian', 'Caucasian', 'Caucasian', 'Caucasian', 'Caucasian', None
$ gender                         <str> 'M', 'F', 'F', 'F', 'M', 'M', 'F', 'M', 'M

In [63]:
(
    training_v2
    .schema
)

OrderedDict([('encounter_id', Int64),
             ('patient_id', Int64),
             ('hospital_id', Int64),
             ('hospital_death', Int64),
             ('age', Utf8),
             ('bmi', Utf8),
             ('elective_surgery', Int64),
             ('ethnicity', Utf8),
             ('gender', Utf8),
             ('height', Utf8),
             ('hospital_admit_source', Utf8),
             ('icu_admit_source', Utf8),
             ('icu_id', Int64),
             ('icu_stay_type', Utf8),
             ('icu_type', Utf8),
             ('pre_icu_los_days', Float64),
             ('readmission_status', Int64),
             ('weight', Utf8),
             ('albumin_apache', Utf8),
             ('apache_2_diagnosis', Utf8),
             ('apache_3j_diagnosis', Utf8),
             ('apache_post_operative', Int64),
             ('arf_apache', Utf8),
             ('bilirubin_apache', Utf8),
             ('bun_apache', Utf8),
             ('creatinine_apache', Utf8),
             ('fio2_

In [55]:
(
    training_v2
    .select(cs.temporal())
    .glimpse()
)

ValueError: max() arg is an empty sequence

In [56]:
(
    training_v2
    .select(cs.categorical())
    .glimpse()
)

ValueError: max() arg is an empty sequence

In [58]:
(
    training_v2
    .select(cs.string())
    .glimpse()
)

Rows: 91713
Columns: 177
$ age                           <str> '68', '77', '25', '81', '19', '67', '59', '70', '45', '50'
$ bmi                           <str> '22.73', '27.42', '31.95', '22.64', 'NA', '27.56', '57.45', 'NA', 'NA', '25.71'
$ ethnicity                     <str> 'Caucasian', 'Caucasian', 'Caucasian', 'Caucasian', 'Caucasian', 'Caucasian', 'Caucasian', 'Caucasian', 'Caucasian', None
$ gender                        <str> 'M', 'F', 'F', 'F', 'M', 'M', 'F', 'M', 'M', 'M'
$ height                        <str> '180.3', '160', '172.7', '165.1', '188', '190.5', '165.1', '165', '170.2', '175.3'
$ hospital_admit_source         <str> 'Floor', 'Floor', 'Emergency Department', 'Operating Room', None, 'Direct Admit', 'Operating Room', 'Emergency Department', 'Other Hospital', 'Direct Admit'
$ icu_admit_source              <str> 'Floor', 'Floor', 'Accident & Emergency', 'Operating Room / Recovery', 'Accident & Emergency', 'Accident & Emergency', 'Accident & Emergency', 'Accident & Emer

In [59]:
(
    training_v2
    .select(cs.integer())
    .glimpse()
)

Rows: 91713
Columns: 8
$ encounter_id          <i64> 66154, 114252, 119783, 79267, 92056, 33181, 82208, 120995, 80471, 42871
$ patient_id            <i64> 25312, 59342, 50777, 46918, 34377, 74489, 49526, 50129, 10577, 90749
$ hospital_id           <i64> 118, 81, 118, 118, 33, 83, 83, 33, 118, 118
$ hospital_death        <i64> 0, 0, 0, 0, 0, 0, 0, 0, 1, 0
$ elective_surgery      <i64> 0, 0, 0, 1, 0, 0, 0, 0, 0, 0
$ icu_id                <i64> 92, 90, 93, 92, 91, 95, 95, 91, 114, 114
$ readmission_status    <i64> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
$ apache_post_operative <i64> 0, 0, 0, 1, 0, 0, 0, 0, 0, 0



In [60]:
(
    training_v2
    .select(cs.float())
    .glimpse()
)

Rows: 91713
Columns: 1
$ pre_icu_los_days <f64> 0.541666667, 0.927777778, 0.000694444, 0.000694444, 0.073611111, 0.000694444, 0.000694444, 0.002083333, 0.009027778, 0.060416667



In [61]:
(
    training_v2
    .select(cs.numeric())
    .glimpse()
)

Rows: 91713
Columns: 9
$ encounter_id          <i64> 66154, 114252, 119783, 79267, 92056, 33181, 82208, 120995, 80471, 42871
$ patient_id            <i64> 25312, 59342, 50777, 46918, 34377, 74489, 49526, 50129, 10577, 90749
$ hospital_id           <i64> 118, 81, 118, 118, 33, 83, 83, 33, 118, 118
$ hospital_death        <i64> 0, 0, 0, 0, 0, 0, 0, 0, 1, 0
$ elective_surgery      <i64> 0, 0, 0, 1, 0, 0, 0, 0, 0, 0
$ icu_id                <i64> 92, 90, 93, 92, 91, 95, 95, 91, 114, 114
$ pre_icu_los_days      <f64> 0.541666667, 0.927777778, 0.000694444, 0.000694444, 0.073611111, 0.000694444, 0.000694444, 0.002083333, 0.009027778, 0.060416667
$ readmission_status    <i64> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
$ apache_post_operative <i64> 0, 0, 0, 1, 0, 0, 0, 0, 0, 0



In [62]:
(
    training_v2
    .schema
)

OrderedDict([('encounter_id', Int64),
             ('patient_id', Int64),
             ('hospital_id', Int64),
             ('hospital_death', Int64),
             ('age', Utf8),
             ('bmi', Utf8),
             ('elective_surgery', Int64),
             ('ethnicity', Utf8),
             ('gender', Utf8),
             ('height', Utf8),
             ('hospital_admit_source', Utf8),
             ('icu_admit_source', Utf8),
             ('icu_id', Int64),
             ('icu_stay_type', Utf8),
             ('icu_type', Utf8),
             ('pre_icu_los_days', Float64),
             ('readmission_status', Int64),
             ('weight', Utf8),
             ('albumin_apache', Utf8),
             ('apache_2_diagnosis', Utf8),
             ('apache_3j_diagnosis', Utf8),
             ('apache_post_operative', Int64),
             ('arf_apache', Utf8),
             ('bilirubin_apache', Utf8),
             ('bun_apache', Utf8),
             ('creatinine_apache', Utf8),
             ('fio2_

## SummaryTools

In [15]:
dfSummary(training_v2)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,encounter_id[int64],Mean (sd) : 65606.1 (37795.1)min < med < max:1.0 < 65665.0 < 131051.0IQR (CV) : 65490.0 (1.7),"91,713 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjYuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/P9b71AAAACXBIWXMAAA9hAAAPYQGoP6dpAAACHklEQVR4nO3XvW/aUBSG8XNNgi4ftcVHYWPp2M0SC0u3jv1ju2YHKTNDl04sCAkZI9ygWsRdujZguwq8yfObOTlXRw+RcEVRGHDrgms/ALjE3bUf8BY551pm1qw4/rsoiqf/+Z634GyoNY8emNlzxdm689fa7fv9/tcoijpVlu52u4Nz7sHMjlXmTfNmZme+oC+G6pxrDQaDb2EY9spuPZ1O91mWfep2uz+DIMhfc/6au/M892b2eTqdPg6Hw32Z2fV6/WGxWHyZTCbDRqNROlTVm5mZ7ff7xDn3/V+xnvuP2gzDsDebzZ6iKCp1uNVq1Vsulx/jOP4xHo+TMrN1529gd9t7n49Go19lZpMkaXnv23EcH9/TzdI09fP5vLfdbptmVilUMzOLouhY5ehmZp1Op/Rs3flb2F3He7vZXy/ejV/9kECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokHB3yYfSNPVl//DhcPBmZlmW+c1m037NedXdqu+uO39JX38AlKM35eLHvVYAAAAASUVORK5CYII="">",0(0.0%)
2,patient_id[int64],Mean (sd) : 65537.1 (37811.3)min < med < max:1.0 < 65413.0 < 131051.0IQR (CV) : 65468.0 (1.7),"91,713 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjYuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/P9b71AAAACXBIWXMAAA9hAAAPYQGoP6dpAAAChUlEQVR4nO3YO28aURCG4Tngy+GSXXEJdDQpLaVAckOTLmV+bNp0QbKl1C7SpHJjIaFlERtbrAlprFQR5uwhxp/yPq09zGj1LrLsttutAa9d7dgHAPs4OfYBr5VzrmFmZxXH19vt9v6Q9/zvCPUvnHONXq/3KUmSTpX55XKZOec+V4k18gUxO+JL8i9f7mdDjVxeM7NfFWdj52Nmk3a7PZhMJss0TR9CBvM899PpdDCfz/vOuWXgXt/tdj+madoKnPtjsVisnHNfzCzo7icxzyzq9ude7p2hxnyzbDab06Io3rXb7R+1Wq18yfnY3WVZ+s1mc3F+fv51MBj8DJktiuK0LMv3o9GoXq/Xg2Ipy9Kb2cXl5eW3fr8fGrnd3d29ub6+/jAajfqhuw/xzKzi7Xme+6urq858Pj8zs/BQzewsSZLOZDK5D/1mub297dzc3Lwdj8ffh8NhFjIbO3+g3c3Hx8d66Ox6vT7x3jfH4/FDxbub3vsy9AUxM8uyrBG5O/qZVb3dzBq7frjX36hpmj6ELs+yrGFm1mq1gmdj5w+1O0bM3cfcfcxntgv/noIEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoWEk31+Kc9zH/rBq9XKm5kVReFns1nzJedVd6veHTu/T1+/AewYQ6dAh33wAAAAAElFTkSuQmCC"">",0(0.0%)
3,hospital_id[int64],Mean (sd) : 105.7 (62.9)min < med < max:2.0 < 109.0 < 204.0IQR (CV) : 114.0 (1.7),147 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjYuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/P9b71AAAACXBIWXMAAA9hAAAPYQGoP6dpAAACk0lEQVR4nO3cQW7aQBSA4TclIAOSLUQFOy5QqQuWHKKH7RGKxAm4ATskyzLCjYWx3VV2gZgZT+0H/7dFBivzD5kZOTF1XQvQd9+6vgGgibeubwDPwxgzFpGR5eWXuq7fb71IqGiFMWY8n89/hWE4s7n+dDolxpjft2IlVLRlFIbhbLPZvEdRlD9yYZqmwW63m8VxPBIRQoV/URTli8Xir8Wl43svspmCCoQKFb781e9zJwc0dTdU3zs5oKmvvlG97uSAphrt+n3t5ICm2ExBBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSoQKlTgb6Y84GHz9hFqy3jY3A9CbR8Pm3tAqJ7wsHm72ExBBUKFCoQKFVijPplnPRp72lAdB0ykx4N2yzMfjXkNtSzLoYiExhjbt7CKxXXARPo9aHc4HY1tt9tFHMffjTEni88Oq6oaWlzXiLdQsywbFkXxc7VaDQaDwUM/tA8OsVgPmIj+80ybozHX8SqKIijL8kee539ExOZY7i5voV4ul7cgCCbr9TpfLpfJo9c7zvCwqqphFEWJ5VmmyIudZ7qO1+FwmO33+8n1eh34uD/va9TpdGp18O0yw33P7mdmO15Jknid2L3dTLnMcN+z2yfHdb3XdWKXehvqB5sZ7nt2+9L3dWKXeh9qVxy+2ay/1fq+TuwSoX6i6/VxX9eJXSLUT7zq+rjPCPWOV1of9x0PpUAFQoUKhAoVCBUqECpUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKECoUKFRv8pJU3T4NE3Pp/PgYhIlmXB8Xic/M/rtX621vt2vb5JX/8A59q2le1SB9MAAAAASUVORK5CYII="">",0(0.0%)
4,hospital_death[int64],1. 02. 1,"83,798 (91.4%)7,915 (8.6%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAAuCAYAAAA/ZmtKAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjYuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/P9b71AAAACXBIWXMAAA9hAAAPYQGoP6dpAAABM0lEQVR4nO3bMWrDQBCF4dkgTNzsIgQ6jA6Rw+YQexshtEVs3GyKtCkyA3rGyv/1gi

## pandas-dq

In [9]:
dqr = dq_report(training_v2, target="hospital_death", html=False, csv_engine="pandas", verbose=1)

/var/folders/g7/822123tx4gn6ndvw2jgh5f140000gn/T/ipykernel_23465/2532020492.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  dqr = dq_report(training_v2, target="hospital_death", html=False, csv_engine="pandas", verbose=1)
/var/folders/g7/822123tx4gn6ndvw2jgh5f140000gn/T/ipykernel_23465/2532020492.py:1: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  dqr = dq_report(training_v2, target="hospital_death", html=False, csv_engine="pandas", verbose=1)


,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
encounter_id,int64,0.000000,100,1.000000,131051.000000,Possible ID colum: drop before modeling process.
patient_id,int64,0.000000,100,1.000000,131051.000000,Possible ID colum: drop before modeling process.
hospital_id,int64,0.000000,0,2.000000,204.000000,No issue
hospital_death,int64,0.000000,0,0.000000,1.000000,has 7915 outliers greater than upper bound (0.0) or lower than lower bound(0.0). Cap them or remove them.
age,float64,4.610033,NA,16.000000,89.000000,"4228 missing values. Impute them with mean, median, mode, or a constant value such as 123., has 170 outliers greater than upper bound (109.5) or lower than lower bound(17.5). Cap them or remove them."
bmi,float64,3.738837,NA,14.844926,67.814990,"3429 missing values. Impute them with mean, median, mode, or a constant value such as 123., has 3289 outliers greater than upper bound (46.8625512725) or lower than lower bound(9.709629732500002). Cap them or remove them."
elective_surgery,int64,0.000000,0,0.000000,1.000000,has 16851 outliers greater than upper bound (0.0) or lower than lower bound(0.0). Cap them or remove them.
ethnicity,object,1.521049,0,nan,nan,"1395 missing values. Impute them with mean, median, mode, or a constant value such as 123., 4 rare categories: ['Other/Unknown', 'Hispanic', 'Asian', 'Native American']. Group them into a single category or drop the categories., Mixed dtypes: has 2 different data types: object, float,"
gender,object,0.027259,0,nan,nan,"25 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
height,float64,1.454538,NA,137.200000,195.590000,"1334 missing values. Impute them with mean, median, mode, or a constant value such as 123., has 463 outliers greater than upper bound (200.75000000000003) or lower than lower bound(139.54999999999998). Cap them or remove them."


In [11]:
fdq = Fix_DQ()

In [12]:
X_train_transformed = fdq.fit_transform(training_v2)

2 ID cols will be dropped from further processing: ['encounter_id', 'patient_id']
    1 zero-variance cols will be dropped from further processing: ['readmission_status']


/Users/zacklarsen/mambaforge/envs/wids/lib/python3.10/site-packages/sklearn/base.py:867: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  return self.fit(X, **fit_params).transform(X)


    Dropping weight which has a high correlation with ['bmi']
    Dropping apache_post_operative which has a high correlation with ['elective_surgery', 'apache_3j_diagnosis']
    Dropping gcs_motor_apache which has a high correlation with ['gcs_eyes_apache']
    Dropping paco2_for_ph_apache which has a high correlation with ['paco2_apache']
    Dropping d1_diasbp_noninvasive_max which has a high correlation with ['d1_diasbp_max']
    Dropping d1_diasbp_noninvasive_min which has a high correlation with ['d1_diasbp_min']
    Dropping d1_heartrate_max which has a high correlation with ['heart_rate_apache']
    Dropping d1_mbp_max which has a high correlation with ['d1_diasbp_max', 'd1_diasbp_noninvasive_max']
    Dropping d1_mbp_min which has a high correlation with ['d1_diasbp_min', 'd1_diasbp_noninvasive_min']
    Dropping d1_mbp_noninvasive_max which has a high correlation with ['d1_diasbp_max', 'd1_diasbp_noninvasive_max', 'd1_mbp_max']
    Dropping d1_mbp_noninvasive_min which has a 

/Users/zacklarsen/mambaforge/envs/wids/lib/python3.10/site-packages/numpy/core/_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/Users/zacklarsen/mambaforge/envs/wids/lib/python3.10/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


    6 mixed data type cols will be dropped from further processing: ['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'apache_3j_bodysystem', 'apache_2_bodysystem']


In [13]:
X_train_transformed

,hospital_id,hospital_death,age,bmi,elective_surgery,height,icu_id,icu_stay_type,icu_type,pre_icu_los_days,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_heartrate_max,h1_mbp_invasive_max,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_temp_max,d1_calcium_max,d1_glucose_min,d1_hco3_max,d1_inr_max,d1_lactate_max,d1_platelets_max,d1_potassium_max,d1_potassium_min,h1_potassium_max,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,apache_4a_hospital_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,118,0,68.0,3.166740,0,180.3,92,admit,Rare,0.019260,2.3,113.0,6.220610,-0.169658,0.336472,3.465736,1.255616,1.525,1.386294,-0.098082,1.609438,5.129899,118.0,27.40,-0.422097,40.0,4.203199,5.588185,7.5855,36.0,134.0,37.45,1.707628,0.0,2.714695,3.850148,32.0,68.0,37.0,72.0,4.204693,40.0,3.555348,10.0,0.686549,-2.137045,4.812184,64.0,131.0,73.0,38.65,3.642836,107.0,93.5,68.0,63.0,119.0,4.969813,3.295837,0.196386,0.999228,-2.602830,211.5,190.0,131.0,37.9,8.50,4.700480,19.0,1.208960,0.832909,5.455321,1.609438,3.40,5.95,7.5885,2.151181,5.950252,5.233779,624.071429,551.250000,-0.010126,-0.02929,-0.126265,-0.539112,-0.114718,-0.163916,-0.084359,-0.064414,-0.145164
1,81,0,77.0,3.347093,0,160.0,90,admit,Med-Surg ICU,0.293669,4.9,108.0,5.318169,-0.169658,1.147402,2.302585,0.444686,1.000,0.693147,-0.098082,0.693147,4.983607,120.0,36.90,-0.422097,46.0,3.637586,3.951244,7.4500,33.0,145.0,35.10,1.707628,1.0,2.617396,4.816241,76.5,95.0,31.0,72.0,5.090678,99.0,3.496508,12.0,0.686549,-2.338554,5.416100,147.5,159.0,67.0,36.30,3.586293,107.0,93.5,61.0,48.0,114.0,4.969813,3.465736,1.702112,-1.459145,-2.682421,211.5,190.0,95.0,36.3,8.60,4.859812,27.0,0.832909,1.504077,6.007584,1.648659,3.80,4.20,7.4500,2.134166,3.951244,3.951244,54.800000,51.000000,0.872721,-0.02929,-0.126265,-0.539112,-0.114718,-0.163916,-0.084359,-0.064414,-0.145164
2,118,0,25.0,3.494991,0,172.7,93,admit,Med-Surg ICU,-0.452029,4.9,122.0,6.556821,-0.169658,1.147402,4.119037,1.320422,1.525,1.386294,-0.098082,1.791759,5.844993,102.0,52.75,-0.422097,68.0,4.203199,5.588185,7.5855,37.0,150.0,36.70,1.707628,0.0,3.314186,4.816241,76.5,88.0,48.0,68.0,5.090678,99.0,3.091042,8.0,-1.511020,-0.221512,5.416100,147.5,148.0,105.0,37.00,3.629660,107.0,93.5,88.0,58.0,96.0,4.969813,3.044522,-0.139657,-0.371133,-1.228128,211.5,190.0,148.0,36.7,10.15,5.257495,34.5,1.208960,2.008214,6.007584,1.916923,5.35,5.95,7.5885,2.151181,5.950252,5.233779,624.071429,551.250000,-0.424989,-0.02929,-0.126265,-0.539112,-0.114718,-0.163916,-0.084359,-0.064414,-0.145164
3,118,0,81.0,3.162940,1,165.1,92,admit,Rare,-0.452029,4.9,203.0,7.095918,-0.169658,1.147402,4.119037,1.320422,0.600,1.609438,-0.098082,1.791759,5.225747,114.0,25.90,-0.422097,60.0,3.433987,4.962845,7.3900,4.0,150.0,34.80,1.707628,1.0,2.197225,4.143135,30.0,48.0,42.0,92.0,4.532599,52.0,3.178054,7.0,0.686549,0.624418,5.105945,78.0,158.0,84.0,38.00,3.577948,62.0,44.0,62.0,44.0,100.0,4.532599,2.564949,-1.056771,0.999228,0.954712,136.0,106.0,136.0,35.6,10.15,4.488636,34.5,0.955511,2.008214,5.293305,1.791759,3.50,5.95,7.4400,2.121063,5.823046,4.634729,342.500000,236.666667,-0.262205,-